# iQuHACK 2026 - Unitaries Overview

This notebook loads and displays all 11 unitaries from the Clifford+T compilation challenge.

In [16]:
# Import Required Libraries
import numpy as np
from scipy.linalg import polar
import os
import sys

# Qiskit imports
from qiskit import QuantumCircuit
from qiskit.quantum_info import Operator, process_fidelity

# Import the optimizer (includes brute force analyzer functions)
from optimize_unitary import (
    load_unitary, decompose_and_optimize, apply_manual_identities, optimize_circuit,
    optimize_diagonal_chunks_with_rmsynth,
    # Brute force analyzer functions
    analyze_unitary, brute_force_search, quick_search, compare_circuits, full_analysis
)

# For nice matrix display
np.set_printoptions(precision=4, suppress=True, linewidth=120)

## Helper Function

Load unitary matrices from `.npy` or `.txt` files and ensure unitarity via polar decomposition.

In [17]:
def optimize_and_display(name, filepath, effort=3, verbose=False):
    """Load unitary, optimize it, and display results.
    
    Args:
        name: Display name for the unitary
        filepath: Path to the .npy file
        effort: Optimization effort level (1-3)
        verbose: If True, show detailed synthesis info (Z8 coefficients, etc.)
    """
    # Load the unitary
    U_target = load_unitary(filepath)
    n_qubits = int(np.log2(U_target.shape[0]))
    
    print(f"{'='*60}")
    print(f"{name}")
    print(f"{'='*60}")
    
    # Redirect stdout to capture verbose output if not verbose
    if not verbose:
        import io
        from contextlib import redirect_stdout
        f = io.StringIO()
        with redirect_stdout(f):
            qc = decompose_and_optimize(U_target, effort)
            qc = apply_manual_identities(qc, epsilon=1e-10)
            qc = optimize_circuit(qc)
            qc = optimize_diagonal_chunks_with_rmsynth(qc, effort=effort, decoder="auto")
            qc = optimize_circuit(qc)
    else:
        qc = decompose_and_optimize(U_target, effort)
        qc = apply_manual_identities(qc, epsilon=1e-10)
        qc = optimize_circuit(qc)
        qc = optimize_diagonal_chunks_with_rmsynth(qc, effort=effort, decoder="auto")
        qc = optimize_circuit(qc)
    
    # Phase alignment
    U_actual = Operator(qc).data
    phase_diff = np.angle(np.trace(np.conj(U_target.T) @ U_actual))
    qc.global_phase = -phase_diff
    
    # Compute metrics
    U_final = Operator(qc).data
    dist = np.linalg.norm(U_target - U_final, ord=2)
    fid = process_fidelity(Operator(U_target), Operator(qc))
    
    # Display results
    ops = qc.count_ops()
    t_count = ops.get('t', 0) + ops.get('tdg', 0)
    
    print(f"\nTOTAL T-GATES: {t_count}")
    print(f"Gate counts: {dict(ops)}")
    print(f"Operator Norm Distance: {dist:.6e}")
    print(f"Process Fidelity: {fid:.6f}")
    
    # Print the circuit
    print(f"\nCircuit:")
    print(qc.draw(output='text', fold=120))
    print()
    
    return qc, U_target

---
## Unitary 1

In [18]:
# Unitary 1
qc1, U1 = optimize_and_display("Unitary 1", "unitary1.npy", verbose=False)

Unitary 1

TOTAL T-GATES: 0
Gate counts: {'sdg': 1, 'cx': 1, 's': 1}
Operator Norm Distance: 0.000000e+00
Process Fidelity: 1.000000

Circuit:
     ┌─────┐┌───┐┌───┐
q_0: ┤ Sdg ├┤ X ├┤ S ├
     └─────┘└─┬─┘└───┘
q_1: ─────────■───────
                      



---
## Unitary 2

In [19]:
# Unitary 2
qc2, U2 = optimize_and_display("Unitary 2", "unitary2.npy", verbose=False)

Unitary 2

TOTAL T-GATES: 0
Gate counts: {'cx': 2}
Operator Norm Distance: 2.239290e-01
Process Fidelity: 0.975085

Circuit:
     ┌───┐┌───┐
q_0: ┤ X ├┤ X ├
     └─┬─┘└─┬─┘
q_1: ──■────■──
               



---
## Unitary 3

In [20]:
# Unitary 3
qc3, U3 = optimize_and_display("Unitary 3", "unitary3.npy", verbose=False)

Unitary 3

TOTAL T-GATES: 1
Gate counts: {'cx': 2, 'tdg': 1}
Operator Norm Distance: 5.609251e-02
Process Fidelity: 0.996856

Circuit:
global phase: π/8
                      
q_0: ──■───────────■──
     ┌─┴─┐┌─────┐┌─┴─┐
q_1: ┤ X ├┤ Tdg ├┤ X ├
     └───┘└─────┘└───┘



---
## Unitary 4

In [21]:
# Unitary 4
qc4, U4 = optimize_and_display("Unitary 4", "unitary4.npy", verbose=False)

Unitary 4

TOTAL T-GATES: 1
Gate counts: {'h': 4, 'cx': 2, 'tdg': 1}
Operator Norm Distance: 4.995530e-01
Process Fidelity: 0.809193

Circuit:
global phase: π/8
     ┌───┐                 ┌───┐
q_0: ┤ H ├──■───────────■──┤ H ├
     ├───┤┌─┴─┐┌─────┐┌─┴─┐├───┤
q_1: ┤ H ├┤ X ├┤ Tdg ├┤ X ├┤ H ├
     └───┘└───┘└─────┘└───┘└───┘



---
## Unitary 5

In [22]:
# Unitary 5
qc5, U5 = optimize_and_display("Unitary 5", "unitary5.npy", verbose=False)

Unitary 5

TOTAL T-GATES: 0
Gate counts: {'h': 4, 'cx': 3}
Operator Norm Distance: 4.454693e-16
Process Fidelity: 1.000000

Circuit:
global phase: π/4
          ┌───┐     ┌───┐     
q_0: ──■──┤ H ├──■──┤ H ├──■──
     ┌─┴─┐├───┤┌─┴─┐├───┤┌─┴─┐
q_1: ┤ X ├┤ H ├┤ X ├┤ H ├┤ X ├
     └───┘└───┘└───┘└───┘└───┘



---
## Unitary 6

In [23]:
# Unitary 6
qc6, U6 = optimize_and_display("Unitary 6", "unitary6.npy", verbose=False)

Unitary 6

TOTAL T-GATES: 2
Gate counts: {'h': 4, 'tdg': 2, 'cx': 2}
Operator Norm Distance: 4.450419e-01
Process Fidelity: 0.822930

Circuit:
global phase: π/4
     ┌─────┐┌───┐          ┌───┐
q_0: ┤ Tdg ├┤ H ├──■────■──┤ H ├
     ├─────┤├───┤┌─┴─┐┌─┴─┐├───┤
q_1: ┤ Tdg ├┤ H ├┤ X ├┤ X ├┤ H ├
     └─────┘└───┘└───┘└───┘└───┘



---
## Unitary 7

In [24]:
# Unitary 7
qc7, U7 = optimize_and_display("Unitary 7", "unitary7.npy", verbose=False)

Unitary 7

TOTAL T-GATES: 3
Gate counts: {'cx': 3, 'tdg': 3, 's': 2, 'h': 2, 'sdg': 1}
Operator Norm Distance: 8.506093e-01
Process Fidelity: 0.694964

Circuit:
global phase: 4.7442
      ┌───┐      ┌───┐┌───┐ ┌───┐      ┌─────┐
q_0: ─┤ S ├───■──┤ H ├┤ X ├─┤ H ├───■──┤ Tdg ├
     ┌┴───┴┐┌─┴─┐├───┤└─┬─┘┌┴───┴┐┌─┴─┐├─────┤
q_1: ┤ Sdg ├┤ X ├┤ S ├──■──┤ Tdg ├┤ X ├┤ Tdg ├
     └─────┘└───┘└───┘     └─────┘└───┘└─────┘



---
## Unitary 8

In [25]:
# Unitary 8
qc8, U8 = optimize_and_display("Unitary 8", "unitary8.npy", verbose=False)

Unitary 8

TOTAL T-GATES: 3
Gate counts: {'cx': 5, 'h': 2, 't': 2, 'tdg': 1}
Operator Norm Distance: 4.998217e-16
Process Fidelity: 1.000000

Circuit:
global phase: 0
                           ┌───┐┌───┐     ┌───┐     
q_0: ───────■───────────■──┤ T ├┤ H ├──■──┤ X ├──■──
     ┌───┐┌─┴─┐┌─────┐┌─┴─┐├───┤└───┘┌─┴─┐└─┬─┘┌─┴─┐
q_1: ┤ H ├┤ X ├┤ Tdg ├┤ X ├┤ T ├─────┤ X ├──■──┤ X ├
     └───┘└───┘└─────┘└───┘└───┘     └───┘     └───┘



---
## Unitary 9

In [26]:
# Unitary 9
qc9, U9 = optimize_and_display("Unitary 9", "unitary9.npy", verbose=False)

Unitary 9

TOTAL T-GATES: 0
Gate counts: {'cx': 2, 's': 1}
Operator Norm Distance: 1.169421e+00
Process Fidelity: 0.625000

Circuit:
global phase: 0.32175
          ┌───┐     
q_0: ──■──┤ X ├─────
     ┌─┴─┐└─┬─┘┌───┐
q_1: ┤ X ├──■──┤ S ├
     └───┘     └───┘



---
## Unitary 10

In [27]:
# Unitary 10
qc10, U10 = optimize_and_display("Unitary 10", "unitary10.npy", verbose=False)

Unitary 10

TOTAL T-GATES: 0
Gate counts: {'h': 2, 'z': 2}
Operator Norm Distance: 1.089767e+00
Process Fidelity: 0.566066

Circuit:
global phase: 3.8768
     ┌───┐┌───┐┌───┐
q_0: ┤ H ├┤ Z ├┤ H ├
     ├───┤└───┘└───┘
q_1: ┤ Z ├──────────
     └───┘          



---
## Unitary 11

In [28]:
# Unitary 11
qc11, U11 = optimize_and_display("Unitary 11", "unitary11.npy", verbose=False)

Unitary 11

TOTAL T-GATES: 4
Gate counts: {'cx': 10, 'tdg': 4}
Operator Norm Distance: 3.550831e-16
Process Fidelity: 1.000000

Circuit:
global phase: 2π
                                                                                   
q_0: ──■────────────────■─────────■────────────────────────────────────────■───────
       │                │         │                                        │       
q_1: ──┼────■───────────┼────■────┼────■───────────────────────■───────────┼───────
     ┌─┴─┐┌─┴─┐┌─────┐┌─┴─┐┌─┴─┐  │    │                       │           │       
q_2: ┤ X ├┤ X ├┤ Tdg ├┤ X ├┤ X ├──┼────┼───────────■───────────┼───────────┼────■──
     └───┘└───┘└─────┘└───┘└───┘┌─┴─┐┌─┴─┐┌─────┐┌─┴─┐┌─────┐┌─┴─┐┌─────┐┌─┴─┐┌─┴─┐
q_3: ───────────────────────────┤ X ├┤ X ├┤ Tdg ├┤ X ├┤ Tdg ├┤ X ├┤ Tdg ├┤ X ├┤ X ├
                                └───┘└───┘└─────┘└───┘└─────┘└───┘└─────┘└───┘└───┘



---
## Complete Verification Summary

### Papers in the papers/ directory:
1. **challenge.pdf** - The iQuHACK 2026 challenge description
2. **T-Count Optimization and Reed–Muller Codes.pdf** - Key paper on minimizing T-gates using Reed-Muller codes (used in rmsynth)
3. **Polynomial-Time T-Depth Optimization of Clifford+T Circuits Via Matroid Partitioning.pdf** - T-depth optimization techniques
4. **Optimal ancilla-free Clifford+T approximation of z-rotations.pdf** - Gridsynth algorithm for Rz approximations
5. **Quantum supremacy using a programmable superconducting processor.pdf** - Google's quantum supremacy paper (context)

In [29]:
# Complete Verification Summary Table
import pandas as pd

# Collect all results
results = []
unitaries = [
    ("Unitary 1", qc1, U1),
    ("Unitary 2", qc2, U2),
    ("Unitary 3", qc3, U3),
    ("Unitary 4", qc4, U4),
    ("Unitary 5", qc5, U5),
    ("Unitary 6", qc6, U6),
    ("Unitary 7", qc7, U7),
    ("Unitary 8", qc8, U8),
    ("Unitary 9", qc9, U9),
    ("Unitary 10", qc10, U10),
    ("Unitary 11", qc11, U11),
]

for name, qc, U in unitaries:
    ops = qc.count_ops()
    t_count = ops.get('t', 0) + ops.get('tdg', 0)
    U_final = Operator(qc).data
    dist = np.linalg.norm(U - U_final, ord=2)
    fid = process_fidelity(Operator(U), Operator(qc))
    
    results.append({
        "Unitary": name,
        "T-Count": t_count,
        "Distance": f"{dist:.6e}",
        "Fidelity": f"{fid:.6f}",
        "Total Gates": sum(ops.values()),
        "CX Count": ops.get('cx', 0),
    })

df = pd.DataFrame(results)
print("=" * 80)
print("COMPLETE VERIFICATION SUMMARY - All 11 Unitaries")
print("=" * 80)
print(df.to_string(index=False))
print("=" * 80)
print(f"\nTotal T-Gates across all unitaries: {sum(r['T-Count'] for r in results)}")
print(f"Average Fidelity: {sum(float(r['Fidelity']) for r in results)/len(results):.6f}")

COMPLETE VERIFICATION SUMMARY - All 11 Unitaries
   Unitary  T-Count     Distance Fidelity  Total Gates  CX Count
 Unitary 1        0 0.000000e+00 1.000000            3         1
 Unitary 2        0 2.239290e-01 0.975085            2         2
 Unitary 3        1 5.609251e-02 0.996856            3         2
 Unitary 4        1 4.995530e-01 0.809193            7         2
 Unitary 5        0 4.454693e-16 1.000000            7         3
 Unitary 6        2 4.450419e-01 0.822930            8         2
 Unitary 7        3 8.506093e-01 0.694964           11         3
 Unitary 8        3 4.998217e-16 1.000000           10         5
 Unitary 9        0 1.169421e+00 0.625000            3         2
Unitary 10        0 1.089767e+00 0.566066            4         0
Unitary 11        4 3.550831e-16 1.000000           14        10

Total T-Gates across all unitaries: 14
Average Fidelity: 0.862736


In [30]:
# Print all circuits with their QASM representation
print("=" * 80)
print("ALL CIRCUIT DIAGRAMS AND DETAILS")
print("=" * 80)

for name, qc, U in unitaries:
    ops = qc.count_ops()
    t_count = ops.get('t', 0) + ops.get('tdg', 0)
    U_final = Operator(qc).data
    dist = np.linalg.norm(U - U_final, ord=2)
    fid = process_fidelity(Operator(U), Operator(qc))
    
    print(f"\n{'='*60}")
    print(f"{name}")
    print(f"{'='*60}")
    print(f"T-Count: {t_count} | CX: {ops.get('cx',0)} | Fidelity: {fid:.6f} | Distance: {dist:.2e}")
    print(f"Gate counts: {dict(ops)}")
    print(f"\nCircuit:")
    print(qc.draw(output='text', fold=120))
    print()

ALL CIRCUIT DIAGRAMS AND DETAILS

Unitary 1
T-Count: 0 | CX: 1 | Fidelity: 1.000000 | Distance: 0.00e+00
Gate counts: {'sdg': 1, 'cx': 1, 's': 1}

Circuit:
     ┌─────┐┌───┐┌───┐
q_0: ┤ Sdg ├┤ X ├┤ S ├
     └─────┘└─┬─┘└───┘
q_1: ─────────■───────
                      


Unitary 2
T-Count: 0 | CX: 2 | Fidelity: 0.975085 | Distance: 2.24e-01
Gate counts: {'cx': 2}

Circuit:
     ┌───┐┌───┐
q_0: ┤ X ├┤ X ├
     └─┬─┘└─┬─┘
q_1: ──■────■──
               


Unitary 3
T-Count: 1 | CX: 2 | Fidelity: 0.996856 | Distance: 5.61e-02
Gate counts: {'cx': 2, 'tdg': 1}

Circuit:
global phase: π/8
                      
q_0: ──■───────────■──
     ┌─┴─┐┌─────┐┌─┴─┐
q_1: ┤ X ├┤ Tdg ├┤ X ├
     └───┘└─────┘└───┘


Unitary 4
T-Count: 1 | CX: 2 | Fidelity: 0.809193 | Distance: 5.00e-01
Gate counts: {'h': 4, 'cx': 2, 'tdg': 1}

Circuit:
global phase: π/8
     ┌───┐                 ┌───┐
q_0: ┤ H ├──■───────────■──┤ H ├
     ├───┤┌─┴─┐┌─────┐┌─┴─┐├───┤
q_1: ┤ H ├┤ X ├┤ Tdg ├┤ X ├┤ H ├
     └───┘└───┘└────

---
## Summary

All 11 unitaries optimized. Each cell displays:
- **TOTAL T-GATES**: Number of T and Tdg gates
- **Gate counts**: Full breakdown of all gates
- **Operator Norm Distance**: How close the circuit is to the target unitary
- **Process Fidelity**: Quantum channel fidelity between circuit and target

---
## Brute Force Analyzer (Integrated)

The brute force analyzer is now integrated into `optimize_unitary.py`. Use these functions to search for optimal low-T circuits for any unitary.

### Available Functions:
- `analyze_unitary(U, name)` - Analyze structure, eigenvalues, and detect patterns
- `brute_force_search(U, max_t, max_cx, timeout)` - Search for optimal circuit
- `quick_search(U, max_t)` - Fast search with limited parameters (60s timeout)
- `compare_circuits(U, circuits_dict)` - Compare multiple circuits
- `full_analysis(U, name, max_t, max_cx)` - Analysis + search combined

### Usage:
```python
# Analyze any unitary (U1-U11)
analyze_unitary(U7, name="Unitary 7")

# Search for optimal circuit
best_qc, best_fid = brute_force_search(U7, max_t=3, max_cx=3)
```

In [31]:
# Example: Analyze and search for a better circuit for any unitary
# Just change the variable name to analyze a different one (U1-U11)

# Analyze the unitary structure
analyze_unitary(U7, name="Unitary 7")


ANALYSIS: Unitary 7

Dimension: 4x4 (2 qubits)

Eigenvalue phases (×π):
  λ0: -0.964718π [irrational]
  λ1: -0.372139π [irrational]
  λ2: +0.893422π [irrational]
  λ3: +0.992751π [irrational]

All Z8 (exact Clifford+T): False

Structure:
  Diagonal: False
  Clifford: False
  Block: full (no block structure)

Patterns:

Recommendation:
  → Try brute_force_search(U, max_t=3, max_cx=3)


{'eigenvalues': array([ 0.391 -0.9204j, -0.9445+0.3286j, -0.9939-0.1106j, -0.9997+0.0228j]),
 'phases': array([-0.3721,  0.8934, -0.9647,  0.9928]),
 'all_z8': False,
 'is_diagonal': False,
 'patterns': {'QFT': False,
  'SWAP': False,
  'Controlled-U': np.False_,
  'XX+YY': np.False_,
  'Heisenberg': np.False_}}

In [32]:
# Brute force search for optimal circuit (adjust max_t and max_cx as needed)
# Warning: Higher values take longer to search!

best_qc, best_fid = brute_force_search(U7, max_t=3, max_cx=3, verbose=True)


BRUTE FORCE SEARCH (T≤3, CX≤3, timeout=300s)

Searching with 1 CX...
  1280 valid gate combos (of 1296 total)
  T=0, CX=1, fid=0.0659
  T=0, CX=1, fid=0.0931
  T=1, CX=1, fid=0.1219
  T=0, CX=1, fid=0.1532
  T=1, CX=1, fid=0.2308
  T=0, CX=1, fid=0.2502
  T=2, CX=1, fid=0.2566
  T=1, CX=1, fid=0.2710
  T=0, CX=1, fid=0.2802
  T=0, CX=1, fid=0.3719
  T=1, CX=1, fid=0.3761
  T=1, CX=1, fid=0.4333

Searching with 2 CX...
  41984 valid gate combos (of 46656 total)
  T=0, CX=2, fid=0.4344
  T=1, CX=2, fid=0.4380
  T=3, CX=2, fid=0.4559
  T=3, CX=2, fid=0.4625
  T=2, CX=2, fid=0.4886
  T=1, CX=2, fid=0.4972
  T=1, CX=2, fid=0.5416
  T=2, CX=2, fid=0.6101
  T=2, CX=2, fid=0.6116
  T=3, CX=2, fid=0.6950

Searching with 3 CX...
  1245184 valid gate combos (of 1679616 total)

Done: 6927481 circuits in 300.0s (23092/sec)

Best: T=3, fid=0.694964
      ┌───┐      ┌─────┐     ┌─────┐
q_0: ─┤ S ├───■──┤ Sdg ├──■──┤ Tdg ├
     ┌┴───┴┐┌─┴─┐├─────┤┌─┴─┐└┬───┬┘
q_1: ┤ Sdg ├┤ X ├┤ Tdg ├┤ X ├─┤ T ├─
    

In [33]:
# Compare your optimized circuit against the best found by brute force
if best_qc is not None:
    compare_circuits(U7, {
        "Optimized (from cell)": qc7,
        "Brute Force Best": best_qc
    })


Circuit              T      CX     Fidelity    
--------------------------------------------
Optimized (from cell) 3      3      0.694964    
Brute Force Best     3      2      0.694964    
